In [ ]:
import cv2
import json
import os

from preprocess_data import PreprocessData


def preprocess_data(input_json_file, output_path, frame_width, frame_height):
    dp = PreprocessData()

    # Read the JSON file
    with open(input_json_file, 'r') as json_file:
        data = json.load(json_file)

    for file in data:
        video_path = file['SENTENCE_FILE_PATH']
        output_dir = os.path.join(output_path)
        os.makedirs(output_dir, exist_ok=True)
        video_name = os.path.split(video_path)[-1]
        output_file = os.path.join(output_dir, video_name)

        # Open the input video
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        current_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        current_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Define the codec for the output video
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        output = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Resize the frame to the desired dimensions
            new_frame = dp.resize(frame, frame_width, frame_height)

            # Write the resized frame to the output video
            output.write(new_frame)

        # Release the video capture and writer objects
        cap.release()
        output.release()


input_json_file = '../../Dataset/test/test.json'
output_path = '../../Dataset/test/Resized/'
preprocess_data(input_json_file, output_path, 331, 331)
